In [ ]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import tqdm
import pyvista as pv
from scipy.spatial import cKDTree
from suncalc import get_position
from pybdshadow.analysis import get_timetable
from pybdshadow.utils import make_clockwise
from pyproj import Transformer

In [ ]:
# def bd_preprocess(building):
#     def bd_preprocess_old(buildings, height=''):
#         '''
#         Preprocess building data, so that we can perform shadow calculation.
#         Remove empty polygons and convert multipolygons into polygons.

#         Parameters
#         --------------
#         buildings : GeoDataFrame
#             Buildings.
#         height : string
#             Column name of building height(meter).

#         Return
#         ----------
#         allbds : GeoDataFrame
#             Polygon buildings
#         '''
        
#         buildings['geometry'] = buildings.buffer(0)
#         buildings = buildings[buildings.is_valid].copy()
#         if height!='':
#             # 建筑高度筛选
#             buildings[height] = pd.to_numeric(buildings[height], errors='coerce')
#             buildings = buildings[buildings[height]>0].copy()

#         polygon_buildings = buildings[buildings['geometry'].apply(
#             lambda r:type(r) == shapely.geometry.polygon.Polygon)]
#         multipolygon_buildings = buildings[buildings['geometry'].apply(
#             lambda r:type(r) == shapely.geometry.multipolygon.MultiPolygon)]
#         allbds = []
#         for j in range(len(multipolygon_buildings)):
#             r = multipolygon_buildings.iloc[j]
#             singlebd = gpd.GeoDataFrame()
#             singlebd['geometry'] = list(r['geometry'].geoms)
#             for i in r.index:
#                 if i != 'geometry':
#                     singlebd[i] = r[i]
#             allbds.append(singlebd)
#         allbds.append(polygon_buildings)
#         allbds = pd.concat(allbds)
#         if len(allbds) > 0:
#             allbds = gpd.GeoDataFrame(allbds)
#             allbds['building_id'] = range(len(allbds))
#             allbds['geometry'] = allbds.buffer(0)
#         else:
#             allbds = gpd.GeoDataFrame()
#         allbds.crs = {'init': 'epsg:4326'}
#         return allbds
    
#     building = building.groupby(['height']).apply(lambda r:r.unary_union).reset_index()
#     building.columns = ['height','geometry']
#     building = gpd.GeoDataFrame(building,geometry = 'geometry')
#     building = bd_preprocess_old(building)
#     return building

In [ ]:
# building_data = gpd.read_file('深圳.geojson')
# building_data

In [ ]:
# building = bd_preprocess(building_data)
# building

In [ ]:
# building.to_file("shenzhen_building.geojson", driver="GeoJSON")
building = gpd.read_file('shenzhen_building.geojson')

In [37]:
def generate_mesh(building, resolution=10):
    """
    生成建筑物的3D模型
    
    Parameters
    ----------
    building : GeoDataFrame
        包含建筑物信息的GeoDataFrame
    resolution : float
        网格分辨率(米)
        
    Returns
    -------
    pyvista.PolyData
        建筑物3D模型
    """
    # 创建一个空的组合数据集
    all_points = []
    all_faces = []
    num_p = 0
    
    for i in tqdm.tqdm(range(len(building)), desc='Generating 3D mesh'):
        row = building.iloc[i]
        polygon = row['geometry']
        height = row['height']
        
        # polygon顺时针
        polygon = make_clockwise(polygon)

        # 获取2D多边形的边界
        x, y = polygon.exterior.xy

        # 创建顶部
        base_vertices = np.column_stack([x, y, np.full(len(x), 0)])
        roof_vertices = np.column_stack([x, y, np.full(len(x), height)])

        # 创建屋顶的多边形
        face = [len(roof_vertices)] + list(range(len(roof_vertices)))
        roof = pv.PolyData(roof_vertices, faces=[face]).triangulate().subdivide_adaptive(
            max_edge_len=resolution,
            max_tri_area=(resolution**2))
        
        faces_id = roof.faces.reshape(-1, 4).copy()
        faces_id[:,1:] += num_p
        num_p += len(roof.points)
        
        all_points.append(roof.points)
        all_faces.append(faces_id)

        for i in range(len(base_vertices)-1):
    
            # 创建墙面的顶点
            wall_vertices = np.array([
                base_vertices[i],
                base_vertices[i+1],
                roof_vertices[i+1],
                roof_vertices[i],
            ])
            wall_area = np.linalg.norm(base_vertices[i+1]-base_vertices[i])*height

            # 不使用四边形，直接创建两个三角形
            faces = np.array([[3, 0, 1, 2], [3, 0, 2, 3]])
            wall = pv.PolyData(wall_vertices, faces=faces)

            wall = wall.subdivide_adaptive(
                max_edge_len=resolution,
                max_tri_area=(resolution**2))

            faces_id = wall.faces.reshape(-1, 4).copy()
            faces_id[:,1:]+=num_p
            num_p+=len(wall.points)
            all_points.append(wall.points)
            all_faces.append(faces_id)

    # 合并所有点和面
    combined_polydata = pv.PolyData(np.row_stack(all_points), np.row_stack(all_faces))
    return combined_polydata

In [38]:
combined_polydata, tri_info = generate_mesh(building)

Generating 3D mesh:   0%|          | 189/437390 [00:10<6:34:08, 18.49it/s]


KeyboardInterrupt: 